# Parselmouth 튜토리얼 (초보자용) — Google Colab · Drive 업로드 · 5쌍 WAV+TextGrid

이 노트북은 **슬라이드**: `Parselmouth_Colab_LJSpeech_Tutorial_StepByStep_DriveData.pptx` 와 **Step 번호가 1:1로 대응**합니다.

## 전제
- 로컬 컴퓨터의 `/data` 폴더에 아래 10개 파일이 있습니다.
  - `LJ001-0001.wav` ~ `LJ001-0005.wav`
  - `LJ001-0001.TextGrid` ~ `LJ001-0005.TextGrid`
- 이 `/data` 폴더를 **Google Drive**로 업로드합니다. (예: `MyDrive/parselmouth_tutorial/data/`)
- Colab에서 Drive를 마운트해서 분석합니다.

> 💡 진행 방법: 위에서 아래로 **셀을 순서대로 실행**하세요.  
> 에러가 나면: **바로 위 셀부터** 다시 실행하면 해결되는 경우가 많습니다.


## Step 0) 필요한 패키지 설치

In [ ]:
!pip -q install praat-parselmouth
!pip -q install tgt pandas matplotlib


## Step 1) Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Step 2) Drive에 업로드한 data 폴더 경로 지정

⚠️ 아래 `DATA_DIR`는 여러분이 Drive에 업로드한 위치에 맞게 수정해야 합니다.


In [ ]:
import os

# 예시: MyDrive/parselmouth_tutorial/data
DATA_DIR = '/content/drive/MyDrive/parselmouth_tutorial/data'

print('exists?', os.path.isdir(DATA_DIR))
if os.path.isdir(DATA_DIR):
    print('files:', len(os.listdir(DATA_DIR)))
    print(sorted(os.listdir(DATA_DIR))[:10])


## Step 3) (선택) data를 /content로 복사 (속도 ↑)

In [ ]:
import shutil

LOCAL_DATA = '/content/data'
if os.path.exists(LOCAL_DATA):
    shutil.rmtree(LOCAL_DATA)
shutil.copytree(DATA_DIR, LOCAL_DATA)

print('copied to:', LOCAL_DATA)
print('files:', len(os.listdir(LOCAL_DATA)))
print(sorted(os.listdir(LOCAL_DATA))[:10])


## Step 4) 5쌍 파일이 제대로 있는지 확인

In [ ]:
import glob

wav_files = sorted(glob.glob(os.path.join(LOCAL_DATA, '*.wav')))
tg_files  = sorted(glob.glob(os.path.join(LOCAL_DATA, '*.TextGrid')))

print('wav:', len(wav_files))
print('TextGrid:', len(tg_files))
print('example wav:', wav_files[:2])
print('example tg :', tg_files[:2])


## Step 5) 샘플 1개 재생해보기

In [ ]:
from IPython.display import Audio

wav_path = wav_files[0]
Audio(wav_path)


## Step 6) import & phone tier 자동 선택 함수

In [ ]:
import tgt
import parselmouth
from parselmouth.praat import call
import pandas as pd

def pick_phone_tier(tg):
    candidates = ['phones', 'phone', 'phn', 'Phoneme', 'phoneme']
    names = tg.get_tier_names()
    for c in candidates:
        if c in names:
            return tg.get_tier_by_name(c), c
    raise ValueError(f'No phone tier found. tier names={names}')

print('OK: ready')


## Step 7) TextGrid 열고 tier 이름 확인

In [ ]:
tg_path = tg_files[0]
tg = tgt.read_textgrid(tg_path)
print('tier names:', tg.get_tier_names())

phone_tier, phone_tier_name = pick_phone_tier(tg)
print('using phone tier:', phone_tier_name)


## Step 8) Parselmouth 분석 객체 만들기 (Pitch/Intensity/Formant)

In [ ]:
wav_path = wav_files[0]
sound = parselmouth.Sound(wav_path)

# 초보자 추천: 간단한 pitch
pitch = sound.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=300)

# Praat 메뉴 호출: intensity/formant
intensity = call(sound, 'To Intensity', 100, 0, True)
formant   = call(sound, 'To Formant (burg)', 0, 5, 5500, 0.005, 50)

print('duration:', sound.duration)


## Step 9) 한 phone 구간에서 값 뽑기 (midpoint)

In [ ]:
itv = phone_tier[5]
mid = itv.start_time + (itv.end_time - itv.start_time)/2

dur_ms = (itv.end_time - itv.start_time) * 1000
f0  = call(pitch,     'Get value at time', mid, 'Hertz', 'Linear')
amp = call(intensity, 'Get value at time', mid, 'cubic')
f1  = call(formant,   'Get value at time', 1, mid, 'Hertz', 'Linear')
f2  = call(formant,   'Get value at time', 2, mid, 'Hertz', 'Linear')
f3  = call(formant,   'Get value at time', 3, mid, 'Hertz', 'Linear')

print(itv.text, f'{dur_ms:.1f}ms', f0, amp, f1, f2, f3)


## Step 10) 한 파일(phone 전부)을 표(DataFrame)로 만들기

In [ ]:
rows = []
for itv in phone_tier:
    mid = itv.start_time + (itv.end_time - itv.start_time)/2
    dur_ms = (itv.end_time - itv.start_time) * 1000
    f0  = call(pitch,     'Get value at time', mid, 'Hertz', 'Linear')
    amp = call(intensity, 'Get value at time', mid, 'cubic')
    f1  = call(formant,   'Get value at time', 1, mid, 'Hertz', 'Linear')
    f2  = call(formant,   'Get value at time', 2, mid, 'Hertz', 'Linear')
    f3  = call(formant,   'Get value at time', 3, mid, 'Hertz', 'Linear')
    rows.append([itv.text, itv.start_time, itv.end_time, dur_ms, f0, amp, f1, f2, f3])

df = pd.DataFrame(rows, columns=['phone','start','end','dur_ms','f0','intensity','f1','f2','f3'])
df.head()


## Step 11) 5개 파일 batch: file별 phone feature 합치기

In [ ]:
all_rows = []

for wav_path in wav_files:
    base = os.path.splitext(os.path.basename(wav_path))[0]
    tg_path = os.path.join(LOCAL_DATA, base + '.TextGrid')
    
    tg = tgt.read_textgrid(tg_path)
    phone_tier, _ = pick_phone_tier(tg)
    
    sound = parselmouth.Sound(wav_path)
    pitch = sound.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=300)
    intensity = call(sound, 'To Intensity', 100, 0, True)
    formant   = call(sound, 'To Formant (burg)', 0, 5, 5500, 0.005, 50)
    
    for itv in phone_tier:
        mid = itv.start_time + (itv.end_time - itv.start_time)/2
        dur_ms = (itv.end_time - itv.start_time) * 1000
        f0  = call(pitch,     'Get value at time', mid, 'Hertz', 'Linear')
        amp = call(intensity, 'Get value at time', mid, 'cubic')
        f1  = call(formant,   'Get value at time', 1, mid, 'Hertz', 'Linear')
        f2  = call(formant,   'Get value at time', 2, mid, 'Hertz', 'Linear')
        f3  = call(formant,   'Get value at time', 3, mid, 'Hertz', 'Linear')
        all_rows.append([base, itv.text, itv.start_time, itv.end_time, dur_ms, f0, amp, f1, f2, f3])

df_all = pd.DataFrame(all_rows, columns=['file','phone','start','end','dur_ms','f0','intensity','f1','f2','f3'])
df_all.head()


## Step 12) CSV 저장 (Drive로)

In [ ]:
OUT_CSV = '/content/drive/MyDrive/parselmouth_tutorial/lj5_phone_features.csv'
df_all.to_csv(OUT_CSV, index=False)
print('saved:', OUT_CSV)


## 보너스) pitch track 간단히 보기(검증용)

In [ ]:
times = pitch.xs()
f0 = pitch.selected_array['frequency']

plt.figure(figsize=(10,3))
plt.plot(times, f0)
plt.xlabel('Time (s)'); plt.ylabel('F0 (Hz)')
plt.title('Pitch track (last loaded file)')
plt.show()


# Troubleshooting (빠른 체크)
- **DATA_DIR 경로 오류**: `exists?`가 False → Drive 폴더 위치를 다시 확인  
- **파일 개수 이상**: `wav=5`, `TextGrid=5`가 아니면 업로드/복사 확인  
- **tier 이름 불일치**: `tg.get_tier_names()` 출력 확인 (phones/phn 등)  
- **f0가 NaN/0**: 무성 구간이면 정상 / pitch_floor/ceiling 조정 가능  
- **formant 값 이상**: ceiling(5000/5500) 조정, 모음 구간만 보기  
- **런타임 초기화**: 설치 셀/Drive 마운트 셀을 다시 실행
